In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font',family='Malgun Gothic')   #한글처리
import seaborn as sns
import pandas as pd

In [3]:
df_I10= pd.read_csv('../../datasets/Join_Collection_NSC2_BND_M20.csv')
df_I10.columns

Index(['Unnamed: 0', 'MDCARE_STRT_DT', 'BTH_YYYY', 'OFIJ_TYPE',
       'MCARE_SUBJ_CD', 'OPRTN_YN', 'ED_RC_TOT_AMT'],
      dtype='object')

In [4]:
df_I10_h = df_I10[['MDCARE_STRT_DT','BTH_YYYY','OPRTN_YN','OFIJ_TYPE']]
df_I10_h  #분석할 컬럼들만 추출

,MDCARE_STRT_DT,BTH_YYYY,OPRTN_YN,OFIJ_TYPE
0,20020710,1921LE,0,0
1,20020805,1921LE,0,0
2,20021017,1921LE,0,0
3,20071008,1921LE,0,0
4,20040312,1921LE,0,0
...,...,...,...,...
4624,20051220,2005,0,0
4625,20050614,2005,0,0
4626,20050716,2005,0,0
4627,20060123,2005,0,0


### 상관관계 확인
- 궁금한 점 : 입원기간에 영향을 미치는 컬럼들
- 목표 변수 : '입원기간' -> 분포 : 언발란스
    - 연속형일 경우 : 정규화 확인
    - 범주형일 경우 : 밸런스 여부 확인
- 설명 변수 설정 : 
    - '과거수술횟수' : 과거 수술 횟수가 많을 수록 입원 기간이 길어짐
    - '연령' : : 상대적으로 나이 많으면 발생률이 높은 것으로 보임.
    - '환자통증정도' : 통증정도가 크면 입원기간이 길어짐
    - '통증기간(월)' : 통증기간은 보통 2달이제일 많았으며 분포는 1-2사이가 가장 많았다. 입원기간과 관계가 적다.
    - '수술시간' : 수술시간은 보통 62분에 중간값을 가지고 있었다.
    - '디스크단면적' : 단면적 크기에 따라 입원기간에 영향을 줌

### EDA 전략
- 목표 변수와 각 설명 변수의 관계 확인
- 해당 변수(목표와 설명) 선택

In [ ]:
df_I10_h.corr()

In [ ]:
sns.histplot(data=df_I10_h , x='MDCARE_STRT_DT',kde=True)
plt.show()

In [ ]:
df_I10_h['MDCARE_STRT_DT'].value_counts()

#### 범주형과 연속형(순서형)으로 dtype변형
- BTH_YYYY(출생년도) object -> int형
    - 1920LE를 int형으로 바꾸기 위해서는 LE를 제거하고 '1921'값을 이용해서 분석해야 하는데 그건 정확한 분석이라고 생각하지 않아우선, 1920LE사람들을 제외하고 분석함
- OPRTN_YN(공상 등 구분) int형 -> object

In [ ]:
df_I10_h.info()   # Dtype 확인

In [ ]:
df_I10_h = df_I10_h[~df_I10_h["BTH_YYYY"].str.contains("1921LE", case=False)] #LE 있는 데이터 삭제

In [ ]:
df_I10_h

In [ ]:
df_I10_h["BTH_YYYY"] = df_I10_h["BTH_YYYY"].astype(int)

In [ ]:
df_I10_h["OPRTN_YN"] = df_I10_h["OPRTN_YN"].astype(object)

In [ ]:
df_I10_h.info()  # Dtype 변경확인

### 시각화 확인


#### 변수간에 상관성 확인
- 목표 변수는 정규화 여부 판단
- 목표 변수와 각 설명 변수 간에 관계 확인

In [ ]:
df_I10_h.describe()  #연속형 확인

In [ ]:
df_I10_h.describe().columns.to_list() #연속형 분포도 확인

In [ ]:
sns.histplot(data=df_I10_h, x='BTH_YYYY', kde=True)
plt.title('출생년도')
plt.show() 

In [ ]:
df_I10_h.describe(include=object).columns.to_list()

In [ ]:
sns.countplot(data=df_I10_h, x='OPRTN_YN')
plt.title('수술여부')
plt.text(0, df_I10_h['OPRTN_YN'].value_counts().loc[0], '미수술', ha='center', va='bottom', fontsize=10)
plt.text(1, df_I10_h['OPRTN_YN'].value_counts().loc[9], '수술', ha='center', va='bottom', fontsize=10)
plt.show()

In [ ]:
df_I10_h['OPRTN_YN'].value_counts()

In [ ]:
sns.countplot(data=df_I10_h, x='OFIJ_TYPE')
plt.title('공상 등 구분')
plt.show()

In [ ]:
df_I10_h['OFIJ_TYPE'].value_counts()   # 0: 해당없음 B : 보훈병원의 국비일반(상이처, 무자격자) 또는 국비보험(급여) 1차

#### 연속형 / 범주형

In [ ]:
sns.boxplot(data=df_I10_h, x='OPRTN_YN', y='MDCARE_STRT_DT')
plt.title('수술여부와 요양개시일자의 관계')
plt.show()

In [ ]:
sns.boxplot(data=df_I10_h, x='OFIJ_TYPE', y='MDCARE_STRT_DT')
plt.title('공상 등과 요양개시일자의 관계')
plt.show()

#### 연속형 / 연속형

In [ ]:
sns.scatterplot(data=df_I10_h, x='BTH_YYYY', y='MDCARE_STRT_DT')
plt.xlabel('BTH_YYYY')
plt.ylabel('MDCARE_STRT_DT')
plt.title('출생년도와 요양개시일자의 관계')
plt.show()